# Frequency Counter
The input probes **must** be set to **10x attenuation** and the input IN1 jumber to **HV**.  
The threashold is set inside the bitstream in the frequancy_counter_v1_0 block.

http://antonpotocnik.com/?p=519284

https://redpitaya-knowledge-base.readthedocs.io/en/latest/learn_fpga/4_lessons/FreqCounter.html

In [6]:
import mmap
import os
import time
import numpy as np

os.system('cat /root/freq_counter.bit > /dev/xdevcfg')

axi_gpio_regset = np.dtype([
    ('gpio1_data'   , 'uint32'),
    ('gpio1_control', 'uint32'),
    ('gpio2_data'   , 'uint32'),
    ('gpio2_control', 'uint32')
])

memory_file_handle = os.open('/dev/mem', os.O_RDWR)
axi_mmap = mmap.mmap(fileno=memory_file_handle, length=mmap.PAGESIZE, offset=0x42000000)
axi_numpy_array = np.recarray(1, axi_gpio_regset, buf=axi_mmap)
axi_array_contents = axi_numpy_array[0]

freq = 125000000  #FPGA Clock Frequency Hz

In [2]:
log2_Ncycles = 16
freq_in = 100000
phase_inc = 2.147482*freq_in
Ncycles = 1<<log2_Ncycles

axi_array_contents.gpio2_data = (0x1f & log2_Ncycles) + (int(phase_inc) << 5)
time.sleep(5) #Allow the counter to stabilise

count = axi_array_contents.gpio1_data
print("Counts: ", count, " cycles: ",Ncycles, " frequency: ",freq/(count/Ncycles),"Hz\n")

Counts:  81920138  cycles:  65536  frequency:  99999.8315433 Hz



In [7]:
#Access the second AXI GPIO Interface
axi_mmap1 = mmap.mmap(fileno=memory_file_handle, length=mmap.PAGESIZE, offset=0x41200000)
axi_numpy_array1 = np.recarray(1, axi_gpio_regset, buf=axi_mmap1)
axi_array_contents1 = axi_numpy_array1[0]
axi_array_contents1

(8191, 4294967295, 127, 4294967295)